In [ ]:
'''
Using the multi-label classifier dataset from earlier exercises (categorized-comments.jsonl in the reddit folder), fit a neural network classifier using scikit-learn. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guideline. Report the accuracy, precision, recall, F1-score, and confusion matrix.


In [1]:
#import json library to read data in jsonl file
import json
#import pandas library
import pandas as pd

In [2]:
#check versions of packages
print('pandas version:', pd.__version__)
# print('numpy version:', np.__version__)
# print('scikit-learn version:', sklearn.__version__)
# print('NLTK version:', nltk.__version__)

pandas version: 1.2.4


In [3]:
#read in the data as a dataframe
filename = "/home/arindam/Documents/mygithub/bu_dsc/data/raw/categorized-comments.jsonl"
with open(filename, 'r') as f:
    jsonl_list = list(f)

list1 = []
for obj in jsonl_list:
    res = json.loads(obj)
    list1.append(res)
    
comments = pd.DataFrame(list1)

#display the first few rows of data
comments.head()
# len(list1)


,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [4]:
# print the dimension of the dataframe
print('The dataframe has a dimension of:',comments.shape)
print('It has {} comments'.format(comments.shape[0]))

The dataframe has a dimension of: (606476, 2)
It has 606476 comments


In [5]:
print('The target names are :', comments['cat'].unique())
print('This shows that there are only 3 categories in the total dataset')

The target names are : ['sports' 'science_and_technology' 'video_games']
This shows that there are only 3 categories in the total dataset


In [6]:
#Convert text to lowercase and romove punctuation
#define a function to clean the text
# import the required libraries here
#import regular expressions library
import re

def clean_text(text):
    """
    Remove punctuations and special characters, makes lower case
    Args: text
    Output: text
    """
    text=text.lower() #makes text lowercase
    text=re.sub('\\d|\\W+|_',' ',text) #removes extra white space
    text=re.sub('[^a-zA-Z]'," ", text) #removes any non-alphabetic characters
    return text

In [7]:
#import word tokenizer from NLTK
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords

def tokenize_text(txt):
    """
    Takes in a sentence, tokenizes the words into a list,
    """
    stop_words = stopwords.words('english')
    tokenizer = TreebankWordTokenizer()
    tokens = tokenizer.tokenize(txt)
    return [token for token in tokens if token not in stop_words]

In [8]:
#Apply NLTK's PorterStemmer
#define a function to stem the words
from nltk.stem.porter import PorterStemmer

def porter_stem_text(token_list):

    porter = PorterStemmer()
    return (" ".join (porter.stem(token) for token in token_list))

In [9]:
# Testing the functions
sample_cmnts = comments[:5000]
# txt = "barely than significantly especially is an the ? better surrounded."
# sample_cmnts['cat'].unique()
# creating a dictionary to replace the string values to numeric
d = {'sports':1,'science_and_technology':2,'video_games':3}
sample_cmnts['ncat'] = sample_cmnts['cat'].map(d)
sample_cmnts

,cat,txt,ncat
0,sports,Barely better than Gabbert? He was significant...,1
1,sports,Fuck the ducks and the Angels! But welcome to ...,1
2,sports,Should have drafted more WRs.\n\n- Matt Millen...,1
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg),1
4,sports,No!! NOO!!!!!,1
...,...,...,...
4995,science_and_technology,I just recently bought a tv with chromecast bu...,2
4996,science_and_technology,"SMS just works, is reliable. Everyone I know h...",2
4997,science_and_technology,[deleted],2
4998,science_and_technology,"It's not perfect, but certainly better than th...",2


In [10]:
# Cleaning and tokenizing the texts in the comments
# Using the transformed column for the model

sample_cmnts['cleaned']=sample_cmnts['txt'].apply(clean_text)
sample_cmnts['tokenized']=sample_cmnts['cleaned'].apply(tokenize_text)
sample_cmnts['stemmed']=sample_cmnts['tokenized'].apply(porter_stem_text)
sample_cmnts

,cat,txt,ncat,cleaned,tokenized,stemmed
0,sports,Barely better than Gabbert? He was significant...,1,barely better than gabbert he was significantl...,"[barely, better, gabbert, significantly, bette...",bare better gabbert significantli better year ...
1,sports,Fuck the ducks and the Angels! But welcome to ...,1,fuck the ducks and the angels but welcome to a...,"[fuck, ducks, angels, welcome, new, niners, fans]",fuck duck angel welcom new niner fan
2,sports,Should have drafted more WRs.\n\n- Matt Millen...,1,should have drafted more wrs matt millen probably,"[drafted, wrs, matt, millen, probably]",draft wr matt millen probabl
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg),1,done https i imgur com yz pm jpg,"[done, https, imgur, com, yz, pm, jpg]",done http imgur com yz pm jpg
4,sports,No!! NOO!!!!!,1,no noo,[noo],noo
...,...,...,...,...,...,...
4995,science_and_technology,I just recently bought a tv with chromecast bu...,2,i just recently bought a tv with chromecast bu...,"[recently, bought, tv, chromecast, built, idea...",recent bought tv chromecast built idea cast vl...
4996,science_and_technology,"SMS just works, is reliable. Everyone I know h...",2,sms just works is reliable everyone i know has...,"[sms, works, reliable, everyone, know, sms, re...",sm work reliabl everyon know sm reach swiftli ...
4997,science_and_technology,[deleted],2,deleted,[deleted],delet
4998,science_and_technology,"It's not perfect, but certainly better than th...",2,it s not perfect but certainly better than the...,"[perfect, certainly, better, official, design]",perfect certainli better offici design


In [11]:
comments['cat'].unique()
# creating a dictionary to replace the string values to numeric
d = {'sports':1,'science_and_technology':2,'video_games':3}
comments['ncat'] = comments['cat'].map(d)
comments

,cat,txt,ncat
0,sports,Barely better than Gabbert? He was significant...,1
1,sports,Fuck the ducks and the Angels! But welcome to ...,1
2,sports,Should have drafted more WRs.\n\n- Matt Millen...,1
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg),1
4,sports,No!! NOO!!!!!,1
...,...,...,...
606471,video_games,No. It's probably only happened to you,3
606472,video_games,I think most of the disappointment came from t...,3
606473,video_games,"dishonored 1/2 looked like arse, so what the h...",3
606474,video_games,[removed],3


In [12]:
# Get the target name
from sklearn.model_selection import train_test_split

# Creating the features from the data set
features, target = sample_cmnts.stemmed, sample_cmnts.ncat

In [13]:
# Make test and training split (20:80)
features_train,features_test,target_train, target_test = train_test_split(features,target, random_state=0, test_size = 0.2)

print('Features-Training Set: ',len(features_train))
print('Features-Test Set: ',len(features_test))
print('Target: Training Set: ',len(target_train))
print('Target: Test Set: ',len(target_test))

Features-Training Set:  4000
Features-Test Set:  1000
Target: Training Set:  4000
Target: Test Set:  1000


In [14]:

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer

classifier = Pipeline([('vect', CountVectorizer()),
                       ('tfidf', TfidfTransformer()),
                       ('ann', MLPClassifier(hidden_layer_sizes=[500,150], verbose=True))
])
clf = classifier.fit(features_train, target_train)

Iteration 1, loss = 0.58971857
Iteration 2, loss = 0.37839883
Iteration 3, loss = 0.18761950
Iteration 4, loss = 0.09082355
Iteration 5, loss = 0.06428572
Iteration 6, loss = 0.05568206
Iteration 7, loss = 0.05156994
Iteration 8, loss = 0.04951650
Iteration 9, loss = 0.05007860
Iteration 10, loss = 0.05006858
Iteration 11, loss = 0.04898457
Iteration 12, loss = 0.04651422
Iteration 13, loss = 0.04777093
Iteration 14, loss = 0.04771221
Iteration 15, loss = 0.04837100
Iteration 16, loss = 0.04761992
Iteration 17, loss = 0.04742211
Iteration 18, loss = 0.04543751
Iteration 19, loss = 0.04580425
Iteration 20, loss = 0.04609289
Iteration 21, loss = 0.04612687
Iteration 22, loss = 0.04538598
Iteration 23, loss = 0.04511230
Iteration 24, loss = 0.04534189
Iteration 25, loss = 0.04674193
Iteration 26, loss = 0.04484787
Iteration 27, loss = 0.04848498
Iteration 28, loss = 0.04670607
Iteration 29, loss = 0.04502825
Iteration 30, loss = 0.04468233
Iteration 31, loss = 0.04455459
Iteration 32, los

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer

classifier = Pipeline([('vect', CountVectorizer()),
                       ('tfidf', TfidfTransformer()),
                       ('ann', MLPClassifier(hidden_layer_sizes=[500,150,100],  max_iter=50, activation='relu',solver='adam',verbose=True))
])
clf = classifier.fit(features_train, target_train)

Iteration 1, loss = 0.57647814
Iteration 2, loss = 0.32844515
Iteration 3, loss = 0.12928790
Iteration 4, loss = 0.06885031
Iteration 5, loss = 0.05745424
Iteration 6, loss = 0.05211757
Iteration 7, loss = 0.04830122
Iteration 8, loss = 0.04998007
Iteration 9, loss = 0.05045178
Iteration 10, loss = 0.04989999
Iteration 11, loss = 0.04849351
Iteration 12, loss = 0.04775651
Iteration 13, loss = 0.04593955
Iteration 14, loss = 0.04610268
Iteration 15, loss = 0.04722192
Iteration 16, loss = 0.04523295
Iteration 17, loss = 0.04622596
Iteration 18, loss = 0.04493510
Iteration 19, loss = 0.04676666
Iteration 20, loss = 0.04523932
Iteration 21, loss = 0.04700011
Iteration 22, loss = 0.04746295
Iteration 23, loss = 0.04408354
Iteration 24, loss = 0.04436455
Iteration 25, loss = 0.04499572
Iteration 26, loss = 0.04465129
Iteration 27, loss = 0.04425375
Iteration 28, loss = 0.04419968
Iteration 29, loss = 0.04418524
Iteration 30, loss = 0.04554814
Iteration 31, loss = 0.04416048
Iteration 32, los

In [18]:
# Predicting the test set for the classifier
y_pred = clf.predict(features_test)
# y_pred

In [19]:
# Displaying the result metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

print(confusion_matrix(target_test,y_pred))
print(classification_report(target_test,y_pred))
print(accuracy_score(target_test, y_pred))

[[635  62]
 [ 71 232]]
              precision    recall  f1-score   support

           1       0.90      0.91      0.91       697
           2       0.79      0.77      0.78       303

    accuracy                           0.87      1000
   macro avg       0.84      0.84      0.84      1000
weighted avg       0.87      0.87      0.87      1000

0.867
